# Imports and parameters

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
import pandas as pd
from download import download_data_and_parse_it
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json
# from tensorflow.keras.preprocessing.sequence import pad_sequences

batch_size = 64
num_classes = 3
learning_rate = 0.001
num_epochs = 3

# Create the model

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 128),
            nn.ReLU(),
            nn.Linear(128, 10)  # Assuming 10 classes
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

class QuickDrawDataset(Dataset):
    def __init__(self, drawings, labels):
        self.drawings = drawings
        self.labels = labels
        
    def __len__(self):
        return len(self.drawings)
    
    def __getitem__(self, idx):
        # Convert the drawing format to image
        drawing = self.drawings.iloc[idx] if isinstance(self.drawings, pd.Series) else self.drawings[idx]
        image = self.drawing_to_image(drawing)
        label = self.labels.iloc[idx] if isinstance(self.labels, pd.Series) else self.labels[idx]
        return torch.FloatTensor(image).unsqueeze(0), label
    
    def drawing_to_image(self, drawing):
        # Create a blank 64x64 image
        image = np.zeros((64, 64))
        
        # For each stroke in the drawing
        for stroke in drawing:
            # Get x and y coordinates
            x = stroke[0]
            y = stroke[1]
            
            # Draw lines between consecutive points
            for i in range(len(x)-1):
                x1, y1 = int(x[i]), int(y[i])
                x2, y2 = int(x[i+1]), int(y[i+1])
                
                # Ensure points are within bounds
                x1 = max(0, min(x1, 63))
                y1 = max(0, min(y1, 63))
                x2 = max(0, min(x2, 63))
                y2 = max(0, min(y2, 63))
                
                # Draw line
                image[y1, x1] = 255
                image[y2, x2] = 255
        
        return image / 255.0

Using cuda device


# Data

In [9]:
le = LabelEncoder()
sword_dataset = download_data_and_parse_it("sword.ndsjon")
tent_dataset = download_data_and_parse_it("tent.ndsjon")
eiffel_dataset = download_data_and_parse_it("star.ndsjon")
df = pd.concat([sword_dataset, tent_dataset])
df = pd.concat([df,eiffel_dataset])
df['class'] = le.fit_transform(df['word'])
df

The file exists.
         word countrycode                      timestamp  recognized  \
123792  sword          MY  2017-03-26 21:14:01.94413 UTC        True   
123793  sword          PL  2017-03-27 14:06:53.35221 UTC        True   
123794  sword          US  2017-03-17 18:29:46.34852 UTC        True   
123795  sword          US    2017-01-27 14:54:45.687 UTC        True   
123796  sword          SE   2017-01-27 09:38:24.6558 UTC        True   
123797  sword          PL  2017-01-30 20:47:58.83752 UTC        True   
123798  sword          DE  2017-01-27 13:56:28.54679 UTC        True   
123799  sword          CA  2017-03-28 13:42:02.27022 UTC        True   
123800  sword          DE  2017-03-12 09:35:38.31056 UTC        True   
123801  sword          AU  2017-03-03 06:18:46.69524 UTC        True   

                  key_id                                            drawing  
123792  5927313763991552  [[[13, 30, 66], [52, 0, 59]], [[11, 26, 53, 76...  
123793  6323617040171008  [[[38, 4

,word,countrycode,timestamp,recognized,key_id,drawing,class
0,sword,SK,2017-03-03 19:28:59.86908 UTC,False,5383244671352832,"[[[108, 99, 82, 40, 11, 0, 16, 39, 65, 144, 19...",1
1,sword,US,2017-03-24 08:34:36.84209 UTC,True,5081784691720192,"[[[42, 1, 0, 6, 22, 55, 78, 111, 107, 92, 77, ...",1
2,sword,PL,2017-03-05 14:53:23.52547 UTC,True,5440421423480832,"[[[53, 74], [0, 174]], [[37, 43, 72, 81], [161...",1
3,sword,DK,2017-03-16 12:31:18.62577 UTC,True,5041545646440448,"[[[41, 1, 5, 98, 134], [173, 171, 174, 172, 17...",1
4,sword,US,2017-03-12 03:13:54.92132 UTC,True,6115505926569984,"[[[95, 92, 64, 54], [0, 24, 153, 255]], [[0, 1...",1
...,...,...,...,...,...,...,...
137614,star,US,2017-03-17 00:47:57.91579 UTC,True,5290529195556864,"[[[63, 76, 85, 96, 105, 107, 111, 119, 129, 13...",0
137615,star,DE,2017-03-29 19:14:10.11319 UTC,True,5098528957267968,"[[[50, 107, 110, 110, 114, 119, 145, 154, 255,...",0
137616,star,FI,2017-01-29 14:59:06.09068 UTC,True,5172681353723904,"[[[5, 14, 61, 92, 137, 145, 158, 156, 153, 148...",0
137617,star,US,2017-03-23 03:38:30.79912 UTC,True,6543687272103936,"[[[100, 117, 141, 152, 255, 254, 166, 211, 196...",0


In [11]:
X = df[["drawing"]].to_numpy()
y = df["class"].to_numpy()

X_shape = X.shape
y_shape = y.shape

print(X_shape, y_shape)

# X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=2/3, random_state=42)
# print(X_temp.dtype)
# print(X_temp[:5])

(392948, 1) (392948,)


In [17]:
# Load the JSON file into a DataFrame
sword_dataset = download_data_and_parse_it("sword.ndsjon")
tent_dataset = download_data_and_parse_it("tent.ndsjon")
eiffel_dataset = download_data_and_parse_it("star.ndsjon")
df = pd.concat([sword_dataset, tent_dataset])
df = pd.concat([df,eiffel_dataset])
df['class'] = le.fit_transform(df['word'])

# Split into training and testing sets
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create datasets
train_dataset = QuickDrawDataset(train_df['drawing'], train_df['class'])
test_dataset = QuickDrawDataset(test_df['drawing'], test_df['class'])

# Create DataLoaders
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define model, criterion, and optimizer (same as before)
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


The file exists.
         word countrycode                      timestamp  recognized  \
123792  sword          MY  2017-03-26 21:14:01.94413 UTC        True   
123793  sword          PL  2017-03-27 14:06:53.35221 UTC        True   
123794  sword          US  2017-03-17 18:29:46.34852 UTC        True   
123795  sword          US    2017-01-27 14:54:45.687 UTC        True   
123796  sword          SE   2017-01-27 09:38:24.6558 UTC        True   
123797  sword          PL  2017-01-30 20:47:58.83752 UTC        True   
123798  sword          DE  2017-01-27 13:56:28.54679 UTC        True   
123799  sword          CA  2017-03-28 13:42:02.27022 UTC        True   
123800  sword          DE  2017-03-12 09:35:38.31056 UTC        True   
123801  sword          AU  2017-03-03 06:18:46.69524 UTC        True   

                  key_id                                            drawing  
123792  5927313763991552  [[[13, 30, 66], [52, 0, 59]], [[11, 26, 53, 76...  
123793  6323617040171008  [[[38, 4

In [18]:
# Training loop
def train_model():
    model.train()
    for epoch in range(10):  # Number of epochs
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

# Testing loop
def test_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total:.2f}%")


In [19]:
# Training and testing (same as before)
train_model()
test_model()

Epoch 1, Loss: 0.7173843969157153
Epoch 2, Loss: 0.6782787286157717
Epoch 3, Loss: 0.6629474310246858
Epoch 4, Loss: 0.6495720981483739
Epoch 5, Loss: 0.6355129301499466
Epoch 6, Loss: 0.6206532851383333
Epoch 7, Loss: 0.6046861752515799
Epoch 8, Loss: 0.5902160903930372
Epoch 9, Loss: 0.5765216335994024
Epoch 10, Loss: 0.5635450629863269
Accuracy: 70.42%
